### 문제 정의

In [1]:
import pandas as pd
data = pd.read_csv("외판원순회문제.csv", index_col = 0)
display(data)

,S,A,B,C,D,E,F,G,H,I,J
S,0,14,17,23,21,18,17,3,25,15,22
A,14,0,2,3,9,16,18,27,1,22,6
B,17,2,0,6,20,10,15,23,5,28,10
C,23,3,6,0,27,12,12,3,29,23,1
D,21,9,20,27,0,15,5,19,22,29,11
E,18,16,10,12,15,0,12,15,15,20,10
F,17,18,15,12,5,12,0,14,24,9,8
G,3,27,23,3,19,15,14,0,27,15,26
H,25,1,5,29,22,15,24,27,0,9,24
I,15,22,28,23,29,20,9,15,9,0,28


In [2]:
print(data.loc["H", "J"] + data.loc["J", "B"])

34


### 유전 알고리즘 연산자 정의

#### 해 표현 및 초기 해 집단 생성

In [12]:
import numpy as np
def initialize(n, points):
    X = [np.random.permutation(points) for _ in range(n)]
    X = np.array(X)
    return X

#### 적합도 함수

In [13]:
def fitness(data, x):
    score = data.loc["S", x[0]]
    for i in range(1, len(x)-1):
        score += data.loc[x[i], x[i+1]]
    score += data.loc[x[-1], "S"]
    return 1/score

#### 선택 연산자

In [14]:
def selection(X, S, k):
    selected_index = []
    _S = S.copy()
    for _ in range(k):
        probs = _S / _S.sum()
        x_idx = np.random.multinomial(1, probs).argmax()
        selected_index.append(x_idx)
        _S[x_idx] = 0
    return X[selected_index]

#### 교차 연산자

In [20]:
def crossover(x1, x2):
    start_idx = np.random.choice(range(0, len(x1)))
    end_idx = np.random.choice(range(start_idx+1, len(x1) + 1))
    new_x = np.empty(len(x1), dtype = object)
    new_x[start_idx:end_idx] = x1[start_idx:end_idx]
    new_x[~np.isin(x1, x1[start_idx:end_idx])] = x2[~np.isin(x2, x1[start_idx:end_idx])]
    return new_x                      

#### 돌연변이 연산자

In [21]:
def mutation(x):
    a, b = np.random.choice(range(len(x)), 2, replace = False)
    (x[b], x[a]) = (x[a], x[b])
    return x

### 메인

In [35]:
def main(n, data, points, k, q, num_generation):
    best_score = -1
    X = initialize(n, points) # 초기해 생성
    for _ in range(num_generation):
        # 해 평가
        S = np.array([fitness(data, x) for x in X]) 
        current_best_score = S.max()
        current_best_x = X[S.argmax()]
        
        # 최고 해 업데이트
        if current_best_score > best_score:
            best_score = current_best_score
            best_x = current_best_x
        
        # k개 해 선택
        X_new = selection(X, S, k)
        
        # 교배 및 돌연변이 연산
        children = []
        for _ in range(n - k):
            parent_idx = np.random.choice(range(k), 2, replace = False)
            child = crossover(X_new[parent_idx[0]], X_new[parent_idx[1]])
            if np.random.random() < q:
                child = mutation(child)
            X_new = np.vstack([X_new, child])
        X = X_new
    return best_x, best_score

In [52]:
n = 100
num_generation = 100
points = data.columns[1:]
k = 60
q = 0.05

In [53]:
GA_best_x, GA_best_score = main(n, data, points, k, q, num_generation)
print(GA_best_x, 1/GA_best_score)

['G' 'I' 'F' 'D' 'A' 'H' 'B' 'C' 'J' 'E'] 67.0


In [54]:
# 임의 탐색
RS_best_score = -1
for x in initialize(10000, data.columns[1:]):
    score = fitness(data, x)
    if score > RS_best_score:
        RS_best_score = score
        RS_best_x = x
        
print(RS_best_x, 1/RS_best_score)

['I' 'H' 'A' 'B' 'J' 'E' 'D' 'F' 'C' 'G'] 76.0
